<a href="https://colab.research.google.com/github/takashinnosuke/study_Keras/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#VGG16モデルの畳み込みベースのインスタンス化
#from keras.applications import VGG16
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet',#このモデルを初期化するための重みのチェックポイント
                  include_top=False,#ネットワークの出力側にある全結合層分類器を含めるかどうか．今回は，catとdogの2つだけに分類するだけで，1000個ものクラスは必要ないのでfalseにしている．
                  input_shape=(150, 150, 3))#ネットワークに供給する画像テンソルの形状．

58900480/58889256 [==============================] - 1s 0us/step


In [4]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     